In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
df=pd.read_csv(r"C:\Users\Admin\Documents\SEM 5 CLG\AI\FINAL_DATA2.csv", sep=',', encoding='windows-1252')

In [6]:
df.head(10)

,Name,Job title,Skills,Location
0,Tanny Fomichkin,Locksmith,lock and security mechanism,Épinal
1,Clary Lowbridge,washing machine,all types of washing machines,Meitang
2,Lin Elvins,care taker,taking care of old people and small children,Luoyang
3,Cordula Gowans,web developer,design and build website,Stockholm
4,Accenture,architect,design new buildings,Donji Dobri?
5,Petey Norrey,carpenter,furniture making,Kastélli
6,Alla Drewery,sweeper,cleaning and maintaining house,Wichit
7,kinesis management consultant Pvt.Ltd,Plumber,install and repair pipes,Yanjiang
8,Portie Bellie,barber,hair cutting,Kondoa
9,Jemima Gorst,cleaning worker,cleaning,R?czna


In [7]:
df['Job title']=df['Job title'].str.strip()
df['Job title']=df['Job title'].str.lower()
df['Skills']=df['Skills'].str.strip()
df['Skills']=df['Skills'].str.lower()


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv=TfidfVectorizer(min_df=3, max_features=None,
                   strip_accents='unicode',analyzer='word',token_pattern=r'\w{1,}\S',
                   ngram_range=(1,3),
                   stop_words='english')
df['Job title']=df['Job title'].fillna('')
df['Skills']=df['Skills'].fillna('')
df['Job']=df['Job title']+" "+df['Skills']

In [9]:
tfv_matrix=tfv.fit_transform(df['Job'])

In [10]:
tfv_matrix

<500x275 sparse matrix of type '<class 'numpy.float64'>'
	with 4780 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [12]:
sig[0]

array([0.76311711, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76311711, 0.76311711, 0.76159416, 0.76159416, 0.76159416,
       0.76311711, 0.76159416, 0.76311711, 0.76159416, 0.76159416,
       0.76311711, 0.76159416, 0.76311711, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159416,
       0.76159416, 0.76159416, 0.76159416, 0.76159416, 0.76159

In [13]:
ind=pd.Series(df['Job title']).drop_duplicates(keep='first')
print(ind.index)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  12,  13,  14,
             19,  21,  24,  25,  26,  30,  34,  35,  45,  48,  54,  74,  84,
             95, 134, 149, 157, 163, 167, 188, 198, 227, 236, 242, 254, 279,
            282, 283, 301, 313, 340, 344, 380, 401, 407, 412, 477, 495],
           dtype='int64')


In [14]:
index_=ind
indices=pd.Series(ind.index)
indices.index=index_

In [15]:
indices

Job title
locksmith                      0
washing machine                1
care taker                     2
web developer                  3
architect                      4
carpenter                      5
sweeper                        6
plumber                        7
barber                         8
cleaning worker                9
app developer                 12
pantry cook                   13
taxi driver                   14
shop worker                   19
painter                       21
truck driver                  24
fry/saute cook                25
welding                       26
beautician                    30
accountant                    34
truck                         35
tester                        45
waiter                        48
housekeeper                   54
baker                         74
electrician                   84
delivery boy                  95
line cook                    134
butcher cook                 149
school bus truck             157


In [16]:
def give_rec(title , sig=sig):
    try:
        idx=indices[title]
        sig_scores=list(enumerate(sig[idx]))
        sig_scores=sorted(sig_scores, key=lambda x: x[1],reverse=True)
        sig_scores=sig_scores[1:8]
        customer_indices =[i[0] for i in sig_scores]
        return df.iloc[customer_indices]
        #values.sort_values(by = 'Experience' ,ascending=False)
       # return df.iloc[customer_indices]
        #return values.sort_values(by = 'Experience' ,ascending=False)
    except:
        contain_values=df[df['Job'].str.contains(title)]
        id=contain_values.index
        df.loc[id]
        print(len(id))
        if len(id)<9 :
            for j in range(0,len(id)): 
                if j==0:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[0:len(id)]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[0:len(id)]
               
        else :
             for j in range(4,8): 
                if j==4:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores2[1:3]
                else:
                    idx=id[j]
                    sig_scores2=list(enumerate(sig[idx]))
                    sig_scores2=sorted(sig_scores2, key=lambda x: x[1],reverse=True)
                    sig_scores=sig_scores+sig_scores2[1:3]
        customer_indices=[i[0] for i in sig_scores]
        return df.iloc[customer_indices]
       # print(value.sort_values(by = 'Experience' ,ascending=False))

In [17]:
give_rec('driver')

8


,Name,Job title,Skills,Location,Job
68,Gui Faers,truck driver,"driving skill, excellent hearing and vision",Guohua,"truck driver driving skill, excellent hearing ..."
88,Nadiya Unstead,truck driver,"driving skill, excellent hearing and vision",Guantánamo,"truck driver driving skill, excellent hearing ..."
110,Elna Warrick,truck driver,"driving skill, excellent hearing and vision",Alkmaar,"truck driver driving skill, excellent hearing ..."
217,Phillis Dunning,truck driver,"driving skill, excellent hearing and vision",Rueil-Malmaison,"truck driver driving skill, excellent hearing ..."
496,Cody Gildea,truck driver,"driving skill, excellent hearing and vision",Licuan,"truck driver driving skill, excellent hearing ..."
102,Delcina Dikles,truck,"driving skill, excellent hearing and vision",San Antonio,"truck driving skill, excellent hearing and vision"
157,Regen Fliege,school bus truck,"carefullness,skill, excellent hearing and vision",L?chi,"school bus truck carefullness,skill, excellent..."
57,Maje Dudin,truck,"skill, excellent hearing and vision",Dallas,"truck skill, excellent hearing and vision"


In [18]:
give_rec('cook')

22


,Name,Job title,Skills,Location,Job
74,Pepi Miguet,baker,"cook cakes,creativity",Kebon,"baker cook cakes,creativity"
257,Salomon Hew,baker,"cook cakes,creativity,skilled in using ovens",Changuillo,"baker cook cakes,creativity,skilled in using o..."
398,Calida Scaplehorn,executive cook,department head for food service,Bakung,executive cook department head for food service
13,Pris Spaduzza,pantry cook,cold food items,Hezhai,pantry cook cold food items
326,Cristal Acom,broiler cook,"grilled,boiled,roasted non veg",Avignon,"broiler cook grilled,boiled,roasted non veg"
477,Sibeal Hirtz,soup and sauce cook,all soups and sauce,Eisen,soup and sauce cook all soups and sauce
25,Hermina Renouf,fry/saute cook,expert in fried sauteed items,Alderetes,fry/saute cook expert in fried sauteed items
416,Quentin Gooding,fry/saute cook,expert in fried and sauteed items,Pagelaran,fry/saute cook expert in fried and sauteed items


In [50]:
give_rec('manager')

3


,Name,Job title,Skills,Location,Job
431,Dorise Robilart,kitchen manager,"purchase items for cooking,handles kitchen",Recarei,"kitchen manager purchase items for cooking,han..."
242,Moe Grier,kitchen manager,handles cooking/kitchen,Obsharovka,kitchen manager handles cooking/kitchen
344,Ulla Massey,catering manager,"handles complaint about cooking,watering service",Tejakalapa,catering manager handles complaint about cooki...


In [20]:
give_rec('mechanic')

,Name,Job title,Skills,Location,Job
168,Atkins global,mechanic,excellent knowledge of motor technology,Ballsh,mechanic excellent knowledge of motor technology
169,Kim Cattini,mechanic,excellent knowledge of motor technology,Amos,mechanic excellent knowledge of motor technology
173,Isac Randalston,mechanic,excellent knowledge of motor technology,Pokrovskoye,mechanic excellent knowledge of motor technology
175,Chrystel Ziemke,mechanic,excellent knowledge of motor technology,Zubrzyca Dolna,mechanic excellent knowledge of motor technology
192,Pippo Sweetland,mechanic,excellent knowledge of motor technology,Daji,mechanic excellent knowledge of motor technology
193,Valentine Rice,mechanic,excellent knowledge of motor technology,Jagabaya Dua,mechanic excellent knowledge of motor technology
471,Gabbey Downton,mechanic,excellent knowledge of motor technology,Maymana,mechanic excellent knowledge of motor technology


In [51]:
give_rec('repairing')

6


,Name,Job title,Skills,Location,Job
412,Tamara Kingsland,cooler repairing,"motor exchange,cooling pad repairing etc",Shahrak-e B?kharz,"cooler repairing motor exchange,cooling pad re..."
313,Eccentric solution Pvt.Ltd,automobile repairing,"tyre exchange,puncture,oiling,etc",Borlänge,"automobile repairing tyre exchange,puncture,oi..."
340,valsatech corp,tv repairing,all types of led's and tv,Mpraeso,tv repairing all types of led's and tv
401,Isabeau Freebury,fridge repairing,"repair not cooling issue,defrost system,etc",Santa Justina,"fridge repairing repair not cooling issue,defr..."
282,Germaine Ferrelli,mobile repairing,repair all types of mobile devices,Paldit,mobile repairing repair all types of mobile de...
254,Elwyn Marte,washing machine repairing,repair all types of washing machines,São Miguel do Rio Torto,washing machine repairing repair all types of ...


In [22]:
give_rec('waiter')

,Name,Job title,Skills,Location,Job
211,Marti Mimmack,delivery boy,provides cooked food to doorstep,Swinford,delivery boy provides cooked food to doorstep
279,Godard Ionn,bartender,"serves the drink,food",Laban,"bartender serves the drink,food"
398,Calida Scaplehorn,executive cook,department head for food service,Bakung,executive cook department head for food service
13,Pris Spaduzza,pantry cook,cold food items,Hezhai,pantry cook cold food items
461,Spencer Jacob,pantry cook,cold food items,Shuikou,pantry cook cold food items
134,Elle Darycott,line cook,responsible for daily food and kitchen tasks,Krajan Kidul Rojopolo,line cook responsible for daily food and kitch...
0,Tanny Fomichkin,locksmith,lock and security mechanism,Épinal,locksmith lock and security mechanism


In [34]:
give_rec('electrician')

,Name,Job title,Skills,Location,Job
202,Lindsey Pund,electrician,solve all electrical issues,Quimilí,electrician solve all electrical issues
249,Scarface Segges,electrician,solve all electrical issues,Tanahgrogot,electrician solve all electrical issues
250,Romeo Alessandrelli,electrician,solve all electrical issues,Q?rghonteppa,electrician solve all electrical issues
260,Barr Merrett,electrician,solve all electrical issues,Choroszcz,electrician solve all electrical issues
287,Lonnie Cowlishaw,electrician,solve all electrical issues,Vilcún,electrician solve all electrical issues
298,kaya limited,electrician,solve all electrical issues,Kow?n-?p,electrician solve all electrical issues
316,Sandy Albury,electrician,solve all electrical issues,Laibin,electrician solve all electrical issues


In [23]:
give_rec('sweeper')

,Name,Job title,Skills,Location,Job
285,Charleen Grisley,sweeper,cleaning and maintaining house,Paradise,sweeper cleaning and maintaining house
299,Tadio McCerery,sweeper,cleaning and maintaining house,Perre,sweeper cleaning and maintaining house
314,Pietra Lemon,sweeper,cleaning and maintaining house,Buckland,sweeper cleaning and maintaining house
330,valsatech corp,sweeper,cleaning and maintaining house,Jacinto Arauz,sweeper cleaning and maintaining house
349,Allard Fermin,sweeper,cleaning and maintaining house,Laborde,sweeper cleaning and maintaining house
361,premium,sweeper,cleaning and maintaining house,Ilare,sweeper cleaning and maintaining house
382,Pansie Pembry,sweeper,cleaning and maintaining house,Rancakole,sweeper cleaning and maintaining house
